In [ ]:
import xarray as xr
import sys
import matplotlib.pyplot as plt
import os
import tqdm
from tqdm import tqdm
import tqdm
import tensorflow as tf
import numpy as np
#DisableqdGPU
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from comet_ml import Experiment
import numpy as np
import tensorflow as tf
import albumentations as A
from functools import partial
AUTOTUNE = tf.data.experimental.AUTOTUNE
from dask.diagnostics import ProgressBar
#RMSE2 = abs(concat_dsets2 - concat_dsets2.sel(model ='Ground Truth')).mean(["lat","lon"])
import cartopy.crs as ccrs
import pandas as pd
# Create an experiment with your api key
sys.path.append(r'/nesi/project/niwa00018/ML_downscaling_CCAM/training_GAN')
from src_unet_init_step import *
from paper_experiments.analyse_experiments_src import *
from paper_experiments.post_process_funcs import *
from tensorflow.keras import layers
import pandas as pd
import json
# changed activation function to hyperbolic tangent



import tensorflow as tf
model_dir ='/nesi/project/niwa00018/ML_downscaling_CCAM/training_GAN/paper_experiments/models/'
filepath = '/nesi/project/niwa00018/ML_downscaling_CCAM/training_GAN/ancil_fields/ERA5_eval_ccam_12km.198110_NZ_Invariant.nc'
models = ['NorESM2-MM', 'ACCESS-CM2','EC-Earth3']
means_filepath = "/nesi/project/niwa00018/ML_downscaling_CCAM/Training_CNN/inputs/ERA5/mean_1974_2011.nc"
stds_filepath = "/nesi/project/niwa00018/ML_downscaling_CCAM/Training_CNN/inputs/ERA5/std_1974_2011.nc"
variables = ['q_500', 'q_850', 'u_500', 'u_850', 'v_500', 'v_850', 't_500', 't_850']
frameworks =['perfect', 'imperfect']
output_model_name = 'Intensity_Penalty'
metrics_precomputed = True

for model in models:
    for framework in frameworks:
        # loading the original dataset
        dsets = xr.open_dataset(f'/nesi/project/niwa00018/ML_downscaling_CCAM/training_GAN/paper_experiments/outputs/{output_model_name}/{model}_{output_model_name}_hist_1986_2005_cascaded_{framework}_applied.nc',
                        chunks = {"member": 1})



            # Example usage:
        vegt, orog, he = load_and_normalize_topography_data(filepath)
        # Example usage:
        df_raw, df, y = load_and_concatenate_hist(model)
        stacked_X = normalize_and_stack(df, means_filepath, stds_filepath, variables)
        common_time = stacked_X.time.to_index().intersection(y.time.to_index())
        with ProgressBar():
            dsets = dsets.load()
            y = y.load()
        print("Loaded data..., now computing metrics")
        if not metrics_precomputed:

            validation_metrics = ValidationMetric(dsets.sel(lat = slice(-55, -33.6), lon =slice(165, 182)))
            validation_metrics = validation_metrics(

                            thresh =1)
            print("saved data")
            validation_metrics.to_netcdf(f'/nesi/project/niwa00018/ML_downscaling_CCAM/training_GAN/paper_experiments/outputs/validation_netcdfs/{output_model_name}/{model}_{output_model_name}_hist_1986_2005_cascaded_{framework}_applied_val_metrics.nc')
            print("computed metrics")
        else:
            validation_metrics = xr.open_dataset(
                f'/nesi/project/niwa00018/ML_downscaling_CCAM/training_GAN/paper_experiments/outputs/validation_netcdfs/{output_model_name}/{model}_{output_model_name}_hist_1986_2005_cascaded_{framework}_applied_val_metrics.nc')
            validation_metrics = validation_metrics.sel(lat = slice(-55, -33.6), lon =slice(165, 182))
        print("saved metrics now computing gt")
        validation_metrics_gt = ValidationMetric(y.sel(time = dsets.time).sel(lat = slice(-55, -33.6), lon =slice(165, 182)))
        validation_metrics_gt = validation_metrics_gt(
                        thresh =1)
        validation_metrics_gt = validation_metrics_gt.expand_dims({"experiment":1})
        validation_metrics_gt['experiment'] = (('experiment'), ['Ground Truth'])                                                         
        metrics = xr.concat([ validation_metrics,validation_metrics_gt], dim ="experiment")
        
        #metrics_summary = abs(metrics.isel(member =0).mean("year") - metrics.sel(experiment='Ground Truth').mean("year").isel(member =0)).mean(["lat","lon"])
        #metrics_summary_ensmean = abs(metrics.mean(["year","member"]) - metrics.sel(experiment='Ground Truth').mean(["year","member"])).mean(["lat","lon"])
        #metrics_summary_ensmean.to_netcdf(f'/nesi/project/niwa00018/ML_downscaling_CCAM/training_GAN/paper_experiments/outputs/validation_netcdfs/{output_model_name}/metrics_summary_ensmean_{model}_{output_model_name}_hist_1986_2005_cascaded_{framework}.nc')
        #metrics_summary.to_netcdf(f'/nesi/project/niwa00018/ML_downscaling_CCAM/training_GAN/paper_experiments/outputs/validation_netcdfs/{output_model_name}/metrics_summary_singlemember_{model}_{output_model_name}_hist_1986_2005_cascaded_{framework}.nc')

        # creating RX1DAY Figure

        plot_metric(metrics.isel(member =0), ens_mean = False, metric_name = 'rx1day',
                     vmin =0, vmax =200, cmap ='BrBG', spacing =5, model = model, framework=framework, generic_experiment_name=output_model_name)
        plot_metric(metrics, ens_mean = True, metric_name = 'rx1day',
                     vmin =0, vmax =200, cmap ='BrBG', spacing =5, model = model, framework=framework, generic_experiment_name=output_model_name)
        plot_metric(metrics, ens_mean = True, metric_name = 'cdd',
                     vmin =0, vmax =30, cmap ='BrBG_r', spacing =0.99, model = model, framework=framework, generic_experiment_name=output_model_name)
        plot_metric(metrics.isel(member =0), ens_mean = False,
                     metric_name = 'cdd', vmin =0, vmax =30, cmap ='BrBG_r', spacing =0.99, model = model, framework=framework, generic_experiment_name=output_model_name)
        plot_metric(metrics.isel(member =0), ens_mean = False,
                     metric_name = 'DJF_rainfall', vmin =0, vmax =7, cmap ='BrBG', spacing =0.15, model = model, framework=framework, generic_experiment_name=output_model_name)
        plot_metric(metrics, ens_mean = True, metric_name = 'DJF_rainfall',
                     vmin =0, vmax =7, cmap ='BrBG', spacing =0.15, model = model, framework=framework, generic_experiment_name=output_model_name)
        plot_metric(metrics, ens_mean = True, metric_name = 'JJA_rainfall',
                     vmin =0, vmax =7, cmap ='BrBG', spacing =0.15, model = model, framework=framework, generic_experiment_name=output_model_name)
        plot_metric(metrics.isel(member =0), ens_mean = False,
                     metric_name = 'JJA_rainfall', vmin =0, vmax =7,
                       cmap ='BrBG', spacing =0.15, model = model, framework=framework, generic_experiment_name=output_model_name)



: 

In [ ]:


import tensorflow as tf
model_dir ='/nesi/project/niwa00018/ML_downscaling_CCAM/training_GAN/paper_experiments/models/'
filepath = '/nesi/project/niwa00018/ML_downscaling_CCAM/training_GAN/ancil_fields/ERA5_eval_ccam_12km.198110_NZ_Invariant.nc'
models = ['NorESM2-MM', 'ACCESS-CM2','EC-Earth3']
means_filepath = "/nesi/project/niwa00018/ML_downscaling_CCAM/Training_CNN/inputs/ERA5/mean_1974_2011.nc"
stds_filepath = "/nesi/project/niwa00018/ML_downscaling_CCAM/Training_CNN/inputs/ERA5/std_1974_2011.nc"
variables = ['q_500', 'q_850', 'u_500', 'u_850', 'v_500', 'v_850', 't_500', 't_850']
frameworks =['perfect', 'imperfect']
output_model_name = 'Intensity_Penalty'
metrics_precomputed = True

for model in models:
    for framework in frameworks:
        # loading the original dataset
        dsets = xr.open_dataset(f'/nesi/project/niwa00018/ML_downscaling_CCAM/training_GAN/paper_experiments/outputs/{output_model_name}/{model}_{output_model_name}_hist_1986_2005_cascaded_{framework}_applied.nc',
                        chunks = {"member": 1})



            # Example usage:
        vegt, orog, he = load_and_normalize_topography_data(filepath)
        # Example usage:
        df_raw, df, y = load_and_concatenate_hist(model)
        stacked_X = normalize_and_stack(df, means_filepath, stds_filepath, variables)
        common_time = stacked_X.time.to_index().intersection(y.time.to_index())
        with ProgressBar():
            dsets = dsets.load()
            y = y.load()
        print("Loaded data..., now computing metrics")
        if not metrics_precomputed:

            validation_metrics = ValidationMetric(dsets.sel(lat = slice(-55, -33.6), lon =slice(165, 182)))
            validation_metrics = validation_metrics(

                            thresh =1)
            print("saved data")
            validation_metrics.to_netcdf(f'/nesi/project/niwa00018/ML_downscaling_CCAM/training_GAN/paper_experiments/outputs/validation_netcdfs/{output_model_name}/{model}_{output_model_name}_hist_1986_2005_cascaded_{framework}_applied_val_metrics.nc')
            print("computed metrics")
        else:
            validation_metrics = xr.open_dataset(
                f'/nesi/project/niwa00018/ML_downscaling_CCAM/training_GAN/paper_experiments/outputs/validation_netcdfs/{output_model_name}/{model}_{output_model_name}_hist_1986_2005_cascaded_{framework}_applied_val_metrics.nc')
            validation_metrics = validation_metrics.sel(lat = slice(-55, -33.6), lon =slice(165, 182))
        print("saved metrics now computing gt")
        validation_metrics_gt = ValidationMetric(y.sel(time = dsets.time).sel(lat = slice(-55, -33.6), lon =slice(165, 182)))
        validation_metrics_gt = validation_metrics_gt(
                        thresh =1)
        validation_metrics_gt = validation_metrics_gt.expand_dims({"experiment":1})
        validation_metrics_gt['experiment'] = (('experiment'), ['Ground Truth'])                                                         
        metrics = xr.concat([ validation_metrics,validation_metrics_gt], dim ="experiment")
        
        #metrics_summary = abs(metrics.isel(member =0).mean("year") - metrics.sel(experiment='Ground Truth').mean("year").isel(member =0)).mean(["lat","lon"])
        #metrics_summary_ensmean = abs(metrics.mean(["year","member"]) - metrics.sel(experiment='Ground Truth').mean(["year","member"])).mean(["lat","lon"])
        #metrics_summary_ensmean.to_netcdf(f'/nesi/project/niwa00018/ML_downscaling_CCAM/training_GAN/paper_experiments/outputs/validation_netcdfs/{output_model_name}/metrics_summary_ensmean_{model}_{output_model_name}_hist_1986_2005_cascaded_{framework}.nc')
        #metrics_summary.to_netcdf(f'/nesi/project/niwa00018/ML_downscaling_CCAM/training_GAN/paper_experiments/outputs/validation_netcdfs/{output_model_name}/metrics_summary_singlemember_{model}_{output_model_name}_hist_1986_2005_cascaded_{framework}.nc')

        # creating RX1DAY Figure

        plot_metric(metrics.isel(member =0), ens_mean = False, metric_name = 'rx1day',
                     vmin =0, vmax =200, cmap ='BrBG', spacing =5, model = model, framework=framework, generic_experiment_name=output_model_name)
        plot_metric(metrics, ens_mean = True, metric_name = 'rx1day',
                     vmin =0, vmax =200, cmap ='BrBG', spacing =5, model = model, framework=framework, generic_experiment_name=output_model_name)
        plot_metric(metrics, ens_mean = True, metric_name = 'cdd',
                     vmin =0, vmax =30, cmap ='BrBG_r', spacing =0.99, model = model, framework=framework, generic_experiment_name=output_model_name)
        plot_metric(metrics.isel(member =0), ens_mean = False,
                     metric_name = 'cdd', vmin =0, vmax =30, cmap ='BrBG_r', spacing =0.99, model = model, framework=framework, generic_experiment_name=output_model_name)
        plot_metric(metrics.isel(member =0), ens_mean = False,
                     metric_name = 'DJF_rainfall', vmin =0, vmax =7, cmap ='BrBG', spacing =0.15, model = model, framework=framework, generic_experiment_name=output_model_name)
        plot_metric(metrics, ens_mean = True, metric_name = 'DJF_rainfall',
                     vmin =0, vmax =7, cmap ='BrBG', spacing =0.15, model = model, framework=framework, generic_experiment_name=output_model_name)
        plot_metric(metrics, ens_mean = True, metric_name = 'JJA_rainfall',
                     vmin =0, vmax =7, cmap ='BrBG', spacing =0.15, model = model, framework=framework, generic_experiment_name=output_model_name)
        plot_metric(metrics.isel(member =0), ens_mean = False,
                     metric_name = 'JJA_rainfall', vmin =0, vmax =7,
                       cmap ='BrBG', spacing =0.15, model = model, framework=framework, generic_experiment_name=output_model_name)